In [1]:
import utils
from utils import *
import generation
import pandas as pd
import numpy as np 
import spacy
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import time

In [2]:
nlp = spacy.load("ja_ginza")

/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'ja_ginza' (5.1.3) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
_ = load_dotenv(find_dotenv())
client = OpenAI()

In [3]:
# just for testing
k = 5
target_word = '初め'
target_level = 'beginner'
context_sentence = "十一月の初めだった。"
prompt = f'write {k} {target_level} example sentences in japanese, that must contain the word "{target_word}" used in a similar sense as "{context_sentence}". following are {k} diverse sentences that must use "{target_word}": '
prompt

'write 5 beginner example sentences in japanese, that must contain the word "初め" used in a similar sense as "十一月の初めだった。". following are 5 diverse sentences that must use "初め": '

In [16]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  temperature = 0.2,
  seed = 42,
  n = 2,
  messages=[
    {"role": "system", "content": "Provide sentences in Japanese in a numbered list, without any translation or romaji. Use the provided kanji form of the word."},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='1. 彼との意見が合わず、議論が続いている。\n2. 予定が合わず、会議に参加できなかった。\n3. 彼女の要求と私の能力が合わず、仕事を断らざるを得なかった。\n4. 彼の行動が社会のルールに合わず、非難されている。\n5. この問題は私たちの専門分野には合わないので、他の専門家に相談した方がいい。', role='assistant', function_call=None, tool_calls=None)


In [21]:
generated_text = "".join([choice.message.content for choice in completion.choices])
utils.extract_sentences_with_target_word(target_word, generated_text, nlp)

['彼との意見が合わず、議論が続いている。',
 '予定が合わず、会議に参加できなかった。',
 '彼女の要求と私の能力が合わず、仕事を断らざるを得なかった。',
 '彼の行動が社会のルールに合わず、非難されている。',
 'この問題は私たちの専門分野には合わないので、他の専門家に相談した方がいい。',
 'この二つのパズルのピースは完璧に合っている。',
 '彼の意見と私の意見は全く合わない。',
 '今日の天気予報は外れていて、実際の天気と合わない。',
 '彼女の性格と私の性格はとても合う。',
 'この料理は辛さと甘さが絶妙に合っている。']

In [11]:
generated_text = completion.choices[0].message.content
generated_text
utils.extract_sentences_with_target_word(target_word, generated_text, nlp)

['彼は新しい仕事の初めに緊張していた。',
 '私たちは春の初めに桜を見に行く予定です。',
 '学校の初めは新しい友達を作るチャンスです。',
 '旅行の初めに地図を確認しましょう。',
 '一年の初めに目標を立てることは重要です。']

In [5]:
# run on everything
dev_only = False
log = utils.get_logger()
df_target = pd.read_csv('../../data/targets/target_words.csv')
#df_target_test = df_target[df_target['is_test'] == False]
# then it should be
# df_target_test = df_target[df_target['is_test'] == True]
# and finally (everything)
df_target_test = df_target
k = 5
target_levels = ['beginner', 'intermediate', 'advanced']
target_levels_real = ['N5', 'N3', 'N1']
candidates_len = []
for target_level, target_level_real in zip(target_levels, target_levels_real):
    for i, data in df_target_test.iterrows():

        target_word = data['target_word']
        context_sentence = data['context_sentence']
        ## just call if file does not exists ->
        if os.path.exists(f"/data/enrico_benedetti/nihongoexample/evaluation/outputs/generation/chatgpt/{target_word}_{target_level_real}_.csv"):
            continue
        
        prompt = f'write {k} {target_level} example sentences in japanese, that must contain the word "{target_word}" used in a similar sense as "{context_sentence}". following are {k} diverse sentences that must use "{target_word}": '
        candidates_list = [context_sentence]
        cycle_max = 0
        while len(candidates_list) -1 < k and cycle_max < 1:
            # call the API

                
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature = 1.0,
                n = 2, # give 2 tries
                seed = 42,
                messages=[
                    {"role": "system", "content": "Provide sentences in Japanese in a numbered list, without any translation or romaji."},
                    {"role": "user", "content": prompt}
                    ])
            # get the generated text and get the sentences
            #generated_text = completion.choices[0].message.content
            generated_text = "".join([choice.message.content for choice in completion.choices])
            # debug the gen text for reference
            log.debug(f"{prompt}, {generated_text}")
            # add the generated sentences
            candidates_list += utils.extract_sentences_with_target_word(target_word, generated_text, nlp)
            candidates = pd.DataFrame({'sentence': candidates_list})
            candidates.drop_duplicates(subset='sentence', inplace=True)
            candidates_list = candidates['sentence'].to_list()
            cycle_max += 1
        #end while
        time.sleep(60)
        #save file
        candidates['target_word'] = target_word
        candidates['context_sentence'] = context_sentence
        candidates['target_level'] = target_level_real
        candidates_len.append(len(candidates_list)-1)
        log.info(f"{target_word}, {target_level}, {len(candidates_list)-1}, {cycle_max}")
        if len(candidates_list) -1 < k:
            log.error(f"{target_word}, {target_level} has only {len(candidates_list) -1} sentences out of {k}")
        if not dev_only:
            candidates.to_csv(f"/data/enrico_benedetti/nihongoexample/evaluation/outputs/generation/chatgpt/{target_word}_{target_level_real}_.csv", index=False)
        else:
            candidates.to_csv(f"/data/enrico_benedetti/nihongoexample/evaluation/outputs/experiments/llmjp_{target_word}_{target_level_real}_.csv", index=False)
        

log.debug(f"Avg number of candidates {np.mean(candidates_len)}, min: {np.min(candidates_len)}")


/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity